In [9]:
import yaml
import os
import pyterrier as pt
import pandas as pd
import json
from sqlalchemy import create_engine

In [10]:
BASE_PATH = "/home/jovyan/work/datasets/LongEval-Web"

with open(BASE_PATH + "/metadata.yml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

In [11]:
!pip install --upgrade typing_extensions
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
import pandas as pd
from sqlalchemy import create_engine
from sentence_transformers import SentenceTransformer, util

# DB-Verbindung
DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"
PORT = "5432"
PASSWORD = "dis182425"

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Lade Sentence-BERT Modell
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Schritt 1: Qrels-Datei laden und Mapping erstellen
column_names = ["QueryID", "Spalte2", "DocID", "Relevanz"]
qrels1 = pd.read_csv(
    "/home/jovyan/work/datasets/LongEval-Web/release_2025_p1/French/LongEval Train Collection/qrels/2023-02_fr/qrels_processed.txt",
    sep=" ", header=None, names=column_names
)

query_ids = list(range(1, 80000))
query_doc_map = {}

for query_id in query_ids:
    qrels_qid = qrels1[(qrels1["QueryID"] == query_id) & (qrels1["Relevanz"] > 0)]
    formatted_docs = ["doc" + str(doc_id) for doc_id in qrels_qid["DocID"].tolist()]
    if formatted_docs:
        query_doc_map[str(query_id)] = formatted_docs  # Keys als Strings speichern

# Schritt 2: Funktion zum Vergleich per BERT
def compare_doc_texts_bert(docids, threshold=0.9):
    if not docids:
        raise ValueError("Die Liste der docids ist leer.")

    placeholders = ','.join([f"'{docid}'" for docid in docids])
    query = f"""
        SELECT docid, sub_collection, text_fr
        FROM "Document"
        WHERE docid IN ({placeholders})
          AND sub_collection IN ('2023-02', '2023-04')
    """
    df = pd.read_sql(query, con=engine)

    if df.empty:
        raise ValueError("Keine Daten gefunden.")

    df = df.drop_duplicates(subset=["docid", "sub_collection"])

    df_pivot = df.pivot(index="docid", columns="sub_collection", values="text_fr").dropna()

    embeddings_02 = model.encode(df_pivot["2023-02"].tolist(), convert_to_tensor=True)
    embeddings_04 = model.encode(df_pivot["2023-04"].tolist(), convert_to_tensor=True)

    similarities = util.cos_sim(embeddings_02, embeddings_04).diagonal().tolist()

    df_pivot["similarity"] = similarities
    df_pivot["texts_similar"] = df_pivot["similarity"] >= threshold

    matching_docids = df_pivot[df_pivot["texts_similar"]].index.tolist()

    return df_pivot.reset_index(), matching_docids

# Schritt 3: BERT-Vergleich auf alle relevanten DocIDs anwenden
all_docids = set(docid for docs in query_doc_map.values() for docid in docs)
_, matching_docids = compare_doc_texts_bert(all_docids, threshold=0.9)
matching_docids_set = set(matching_docids)

# Schritt 4: Filtere pro Query und schneide "doc" vorne weg
filtered_query_doc_map = {
    query_id: [docid[3:] for docid in docids if docid in matching_docids_set]
    for query_id, docids in query_doc_map.items()
}

# Ausgabe der gefilterten DocIDs (nur als Zahl)
for qid, docs in filtered_query_doc_map.items():
    print(f"Gefilterte DocIDs für Query {qid} (nur Zahl): {docs}")

Gefilterte DocIDs für Query 3 (nur Zahl): ['1646118']
Gefilterte DocIDs für Query 8 (nur Zahl): ['19754']
Gefilterte DocIDs für Query 12 (nur Zahl): ['20559', '22877']
Gefilterte DocIDs für Query 18 (nur Zahl): ['19416', '18882']
Gefilterte DocIDs für Query 19 (nur Zahl): ['8323']
Gefilterte DocIDs für Query 20 (nur Zahl): ['1716', '4311']
Gefilterte DocIDs für Query 23 (nur Zahl): []
Gefilterte DocIDs für Query 24 (nur Zahl): ['8592']
Gefilterte DocIDs für Query 26 (nur Zahl): ['3344995']
Gefilterte DocIDs für Query 27 (nur Zahl): ['2588', '23258']
Gefilterte DocIDs für Query 28 (nur Zahl): ['16256', '13703']
Gefilterte DocIDs für Query 29 (nur Zahl): ['26747', '2869393']
Gefilterte DocIDs für Query 30 (nur Zahl): ['2874670', '1644643', '955', '23020', '1706936']
Gefilterte DocIDs für Query 32 (nur Zahl): ['2904587', '1688288']
Gefilterte DocIDs für Query 33 (nur Zahl): ['1684120']
Gefilterte DocIDs für Query 41 (nur Zahl): ['21116']
Gefilterte DocIDs für Query 42 (nur Zahl): ['21520'

In [13]:
# Run File einlesen
run_file = "/home/jovyan/work/datasets/LongEval-Web/runs/longeval-web-fr-2023-04-BM25.gz"
run = pt.io.read_results(run_file)

print(run.head())

# Kopiere die Run-Datei und stelle sicher, dass qid und docno Strings sind
reranked_run = run.copy()
reranked_run['qid'] = reranked_run['qid'].astype(str)
reranked_run['docno'] = reranked_run['docno'].astype(str).str.strip().str.lower()

# Normalisiere auch die Keys und Werte im Dictionary
filtered_query_doc_map = {
    str(qid): [doc.strip().lower() for doc in docs]
    for qid, docs in filtered_query_doc_map.items()
}

# Neue Score-Spalte initialisieren
reranked_run['new_score'] = reranked_run['score']

# Gruppiere die Run-Datei einmal nach qid (für schnelleren Zugriff)
qid_groups = dict(tuple(reranked_run.groupby('qid')))

# Boosting pro Query-ID
for i, (query_id, relevant_docs) in enumerate(filtered_query_doc_map.items()):
    if query_id not in qid_groups:
        continue

    qid_df = qid_groups[query_id]
    boost_mask = qid_df['docno'].isin(relevant_docs)
    boost_indices = qid_df[boost_mask].index

    # Boost anwenden
    if not boost_indices.empty:
        reranked_run.loc[boost_indices, 'new_score'] = reranked_run.loc[boost_indices, 'score'] * 3

# Neue Scores übernehmen
reranked_run['score'] = reranked_run['new_score']
reranked_run = reranked_run.drop(columns=['new_score'])

# Sortieren nach Score und neue Ränge vergeben
reranked_run = reranked_run.sort_values(['qid', 'score'], ascending=[True, False])
reranked_run['rank'] = reranked_run.groupby('qid').cumcount() + 1

# Metadaten hinzufügen
reranked_run['iter'] = 0
reranked_run['name'] = "CIR-longeval-web-fr-2023-04-BM25"

# Spalten in gewünschter Reihenfolge
reranked_run = reranked_run[['qid', 'iter', 'docno', 'rank', 'score', 'name']]

# **Hier Datentypen korrekt setzen vor dem Speichern**
reranked_run['qid'] = reranked_run['qid'].astype(str)
reranked_run['iter'] = reranked_run['iter'].astype(int)
reranked_run['docno'] = reranked_run['docno'].astype(str).str.strip().str.lower()
reranked_run['rank'] = reranked_run['rank'].astype(int)
reranked_run['score'] = reranked_run['score'].astype(float)
reranked_run['name'] = reranked_run['name'].astype(str)

# Vorschau der ersten Zeilen
print(reranked_run.head())

# Speichere das Ergebnis im korrekten TREC-Format ohne Header
reranked_run.to_csv(
    "/home/jovyan/work/datasets/LongEval-Web/index/Gruppe_JMFT/2023-04/reranked_run_bert.gz",
    sep='\t',
    index=False,
    header=False,
    compression='gzip'
)


  qid    docno  rank      score       name
0   3  2214755     0  24.198403  pyterrier
1   3   684186     1  23.318612  pyterrier
2   3   637997     2  23.125073  pyterrier
3   3   430968     3  22.952865  pyterrier
4   3   643034     4  22.752546  pyterrier
         qid  iter    docno  rank      score                              name
509134  1000     0     7496     1  51.884904  CIR-longeval-web-fr-2023-04-BM25
508794  1000     0    12254     2  40.162949  CIR-longeval-web-fr-2023-04-BM25
508795  1000     0    27513     3  36.520265  CIR-longeval-web-fr-2023-04-BM25
508796  1000     0  2912830     4  35.808008  CIR-longeval-web-fr-2023-04-BM25
508797  1000     0     6772     5  34.736942  CIR-longeval-web-fr-2023-04-BM25


In [6]:
#Run File einlesen
run_file = "/home/jovyan/work/datasets/LongEval-Web/runs/longeval-web-fr-2023-04-BM25.gz"
run = pt.io.read_results(run_file)


print(run.head())

  qid    docno  rank      score       name
0   3  2214755     0  24.198403  pyterrier
1   3   684186     1  23.318612  pyterrier
2   3   637997     2  23.125073  pyterrier
3   3   430968     3  22.952865  pyterrier
4   3   643034     4  22.752546  pyterrier


In [7]:
# Kopiere die Run-Datei und stelle sicher, dass qid und docno Strings sind
reranked_run = run.copy()
reranked_run['qid'] = reranked_run['qid'].astype(str)
reranked_run['docno'] = reranked_run['docno'].astype(str).str.strip().str.lower()

# Normalisiere auch die Keys und Werte im Dictionary
filtered_query_doc_map = {
    str(qid): [doc.strip().lower() for doc in docs]
    for qid, docs in filtered_query_doc_map.items()
}

# Neue Score-Spalte initialisieren
reranked_run['new_score'] = reranked_run['score']

# Gruppiere die Run-Datei einmal nach qid (für schnelleren Zugriff)
qid_groups = dict(tuple(reranked_run.groupby('qid')))

# Boosting pro Query-ID
for i, (query_id, relevant_docs) in enumerate(filtered_query_doc_map.items()):
    if query_id not in qid_groups:
        continue

    qid_df = qid_groups[query_id]
    boost_mask = qid_df['docno'].isin(relevant_docs)
    boost_indices = qid_df[boost_mask].index

    # Boost anwenden
    if not boost_indices.empty:
        reranked_run.loc[boost_indices, 'new_score'] = reranked_run.loc[boost_indices, 'score'] * 3

# Neue Scores übernehmen
reranked_run['score'] = reranked_run['new_score']
reranked_run = reranked_run.drop(columns=['new_score'])

# Sortieren nach Score und neue Ränge vergeben
reranked_run = reranked_run.sort_values(['qid', 'score'], ascending=[True, False])
reranked_run['rank'] = reranked_run.groupby('qid').cumcount() + 1

# Metadaten hinzufügen
reranked_run['iter'] = 0
reranked_run['name'] = "CIR-longeval-web-fr-2023-04-BM25"

# Spalten in gewünschter Reihenfolge
reranked_run = reranked_run[['qid', 'iter', 'docno', 'rank', 'score', 'name']]

# Vorschau der ersten Zeilen
print(reranked_run.head())

         qid  iter    docno  rank      score                              name
509134  1000     0     7496     1  51.884904  CIR-longeval-web-fr-2023-04-BM25
508794  1000     0    12254     2  40.162949  CIR-longeval-web-fr-2023-04-BM25
508795  1000     0    27513     3  36.520265  CIR-longeval-web-fr-2023-04-BM25
508796  1000     0  2912830     4  35.808008  CIR-longeval-web-fr-2023-04-BM25
508797  1000     0     6772     5  34.736942  CIR-longeval-web-fr-2023-04-BM25


In [8]:
# Speichere das Ergebnis im korrekten TREC-Format ohne Header
reranked_run.to_csv("/home/jovyan/work/datasets/LongEval-Web/index/Gruppe_JMFT/2023-04/reranked_run_bert.gz", sep='\t', index=False, header=False, compression='gzip')